In [22]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('text', usetex = True)
mpl.rc('font', family = 'serif')
import warnings
from scipy.integrate import odeint
import torch
from mlp import MLP
from experiment import *
from data import *
plt.rcParams["figure.figsize"] = [16,9]
# https://journals.aps.org/pre/pdf/10.1103/PhysRevE.54.3545
# some function structures based on https://github.com/erikdao/mlp-timeseries/blob/master/data.py
#warnings.filterwarnings('ignore')

In [16]:
def pendulum(stateIn, t, beta, gamma, wD, w0):
        """
        pendulum state equation
        """
        w, theta = stateIn
        stateOut = np.array([-2*beta*w - w0**2*np.sin(theta) + gamma*w0**2*np.cos(wD*t), w])
        return stateOut
    
def run_pendulum(tmax=10, nt=500, beta=3*np.pi/4, gamma=1.01, wD=2*np.pi, w0=3*np.pi, wInit=0, thetaInit=0):
    t=np.linspace(0,tmax,nt)
    f = odeint(pendulum, (wInit, thetaInit), t, args=(beta, gamma, wD, w0))
    _, theta=f.T
    return t,theta


def generate_dataset(nData=300, train_test_split=0.5, tmax=10, nt=500):
    # create randomly perturbed initial conditions wInit, thetaInit
    wInitMu, wInitSigma = 0, 0.01
    wInitRnd=np.random.normal(wInitMu, wInitSigma, nData)
    thetaInitMu, thetaInitSigma = 0, 0.01
    thetaInitRnd=np.random.normal(thetaInitMu, thetaInitSigma, nData)
    
    # create randomly perturbed values for parameters beta, gamma, wD, w0
    betaMu, betaSigma = 3*np.pi/4, 0.1
    betaRnd=np.random.normal(betaMu, betaSigma, nData)
    gammaMu, gammaSigma = 1.01, 0.03
    gammaRnd=np.random.normal(gammaMu, gammaSigma, nData)
    wDMu, wDSigma = 2*np.pi, 0.1
    wDRnd=np.random.normal(wDMu, wDSigma, nData)
    w0Mu, w0Sigma = 3*np.pi, 0.1
    w0Rnd=np.random.normal(w0Mu, w0Sigma, nData)


    # create dataset with nData "points" (each point is a time series of length nt)
    dataset=np.empty([nData, nt])
    params=np.empty([nData, 6])
    
    for i in range(nData):
        params[i]=np.array([betaRnd[i], gammaRnd[i], wDRnd[i], w0Rnd[i], wInitRnd[i], thetaInitRnd[i]])
        _, theta=run_pendulum(tmax, nt, betaRnd[i], gammaRnd[i], wDRnd[i], w0Rnd[i], wInitRnd[i], thetaInitRnd[i])
        dataset[i]=theta
    return params, dataset
    
    
def split_dataset(inputs, outputs, n_train=80, n_val=100, n_test=120):
    X_train = inputs[:n_train]
    X_val = inputs[n_train:n_val]
    X_test = inputs[n_val:n_test]

    y_train = outputs[:n_train]
    y_val = outputs[n_train:n_val]
    y_test = outputs[n_val:n_test]

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)


def create_dataset():
    params, dataset = generate_dataset()
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = split_dataset(params, dataset)

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

def create_pytorch_loader(X, y, batch_size=100, shuffle=True):
    params = torch.from_numpy(X.astype('float32'))
    data = torch.from_numpy(y.astype('float32'))

    dataset = data_utils.TensorDataset(params, data)
    dataloader = data_utils.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

def create_pytorch_data(batch_size=100):
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = create_dataset()
    train_loader = create_pytorch_loader(X_train, y_train, batch_size=batch_size, shuffle=True)
    val_loader = create_pytorch_loader(X_val, y_val, batch_size=batch_size*2, shuffle=False)
    test_loader = create_pytorch_loader(X_test, y_test, batch_size=batch_size*2, shuffle=False)

    return train_loader, val_loader, test_loader


In [57]:
(X_train, y_train), (X_val, y_val), (X_test, y_test) = create_dataset()

In [11]:
model = MLP()


TypeError: 'NoneType' object is not iterable

In [31]:
dataset=np.empty([5, 10])

In [23]:
grid_search_network()

NameError: name 'create_pytorch_data' is not defined

In [32]:
dataset[0]=np.linspace(0,1,10)

In [33]:
dataset

array([[ 0.00000000e+000,  1.11111111e-001,  2.22222222e-001,
         3.33333333e-001,  4.44444444e-001,  5.55555556e-001,
         6.66666667e-001,  7.77777778e-001,  8.88888889e-001,
         1.00000000e+000],
       [ 4.27197409e+096,  7.29111968e-304,  3.68935022e+019,
         7.29111856e-304,  4.27197409e+096,  7.29111968e-304,
         3.68935022e+019,  7.41841609e-068,  3.68934883e+019,
         7.29112015e-304],
       [ 7.29111856e-304,  7.41841609e-068,  3.68934883e+019,
        -2.87284961e-188,  7.29111922e-304, -2.48104089e-265,
         7.29111856e-304, -2.87284836e-188,  7.29111922e-304,
        -2.48104089e-265],
       [ 3.68935070e+019,  3.68934883e+019,  7.29112040e-304,
         7.29111856e-304,  3.68935070e+019,  3.68934883e+019,
         4.27197625e+096,  7.29111968e-304,  3.68935022e+019,
         7.29111856e-304],
       [ 4.27197409e+096,  7.29111968e-304,  3.68935022e+019,
         2.46006312e+260,  2.46006374e+260,  7.29111856e-304,
         7.29112108e-304